In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from datetime import datetime, timedelta
import matplotlib.colors as clr
from math import ceil
import matplotlib.patches as patches

plt.rcParams['font.sans-serif'] = ['Heiti TC'] #不同系统不一样，我这里是mac系统
%matplotlib

Using matplotlib backend: MacOSX


In [3]:
picture_image = Image.open("apple_store首页.jpg")

# 这里是交互式的，查看对应的位置
fig, ax = plt.subplots()
ax.imshow(picture_image)

In [4]:
location_data = pd.read_csv("按钮位置对照表.csv")#.head(10)
location_data

,location_x,location_y,name,pointer_x,pointer_y
0,0,0,Mac,275,600
1,0,1,iPhone,856,600
2,1,0,iPad,300,1383
3,1,1,Apple Watch,875,1383
4,2,0,AirPods,300,2110
5,2,1,HomePod mini,875,2110
6,3,0,AirTag,300,2870
7,3,1,配件,875,2870
8,4,0,选购,98,3517
9,4,1,课程,330,3517


In [5]:
# create data
faker_data = pd.DataFrame({'name': location_data['name']})

for index, value in enumerate([datetime(2022, 6, 10) + timedelta(days=i) for i in range(20)]):
    faker_data[value] = np.random.randint(10 + index * 42, 1000 * index + 100, size=faker_data.shape[0])

faker_data = faker_data.melt(id_vars=['name'], var_name="date", value_name="data1").reset_index(drop=True)
faker_data

,name,date,data1
0,Mac,2022-06-10,72
1,iPhone,2022-06-10,26
2,iPad,2022-06-10,22
3,Apple Watch,2022-06-10,17
4,AirPods,2022-06-10,55
...,...,...,...
255,选购,2022-06-29,5631
256,课程,2022-06-29,8747
257,为你推荐,2022-06-29,12047
258,搜索,2022-06-29,7250


In [6]:
def custom_func1(x):
    return ceil(x.max() / 20) * 10
sort_index_data = faker_data.groupby(['name']).agg(
    value =('data1', custom_func1)
).reset_index(drop=False).sort_values(by=['value'],ascending=False).reset_index(drop=True).reset_index(drop=False)
sort_index_data

,index,name,value
0,0,课程,8420
1,1,AirPods,7910
2,2,AirTag,7510
3,3,为你推荐,7480
4,4,HomePod mini,7410
5,5,iPad,7040
6,6,购物袋,6990
7,7,iPhone,6900
8,8,搜索,6720
9,9,Apple Watch,6420


In [7]:

cmap = clr.LinearSegmentedColormap.from_list('custom blue', ['#244162','#DCE6F1'], N=location_data.shape[0])
# cmap = clr.LinearSegmentedColormap.from_list('custom blue', ['gray','red'], N=location_data.shape[0])

In [8]:


def annotate_axes(ax, text, fontsize=12):
    ax.text(0.5, 0.5, text, transform=ax.transAxes,
            ha="center", va="center", fontsize=fontsize, color="darkgrey")


fig = plt.figure(constrained_layout=False, facecolor='0.9', figsize=(11, 8))
gs = fig.add_gridspec(nrows=sort_index_data.shape[0], ncols=7, left=0.01, right=0.99,
                      hspace=0, wspace=0.1)

ax0_image = fig.add_subplot(gs[:, :2])
ax0_image.imshow(picture_image)
# annotate_axes(ax0_image, "ax0_image")
ax0_image.axis('off')

part2_ax_list = []
connect_patch_list = []
for index, value in enumerate(location_data['name'].unique().tolist()):
    temp_ax = fig.add_subplot(gs[index, 2:])

    temp_data = faker_data.loc[faker_data['name'] == value].sort_values(by=['date'])

    face_color = cmap(sort_index_data.loc[sort_index_data['name'] == value]['index'].tolist()[0])
    temp_ax.fill_between(range(temp_data['data1'].shape[0]), 0, temp_data['data1'],
                         facecolor=face_color)

    y_ticks_center = ceil(temp_data['data1'].max() / 20) * 10
    temp_ax.set_yticks([0, y_ticks_center])
    temp_ax.set_yticklabels(["0", str(int(y_ticks_center))])
    temp_ax.yaxis.grid(True, linestyle=':')

    location_ax1 = location_data.loc[location_data['name'] == value]

    connectionstyle = "arc3,rad=0.2" if location_ax1['location_x'].tolist()[0] == 1 else "arc3,rad=-0.2"
    if location_ax1['location_x'].tolist()[0] ==4:
        connectionstyle = "arc3,rad=-0.3"


    con = patches.ConnectionPatch(
        xyA=(location_ax1['pointer_x'].tolist()[0], location_ax1['pointer_y'].tolist()[0]),
        xyB=(0, temp_data['data1'].mean()),
        axesA=ax0_image,
        axesB=temp_ax,
        coordsA=ax0_image.transData,
        coordsB=temp_ax.transData,
        arrowstyle="fancy",
        connectionstyle=connectionstyle,
        shrinkA=5, shrinkB=5,
        mutation_scale=20,
        fc="gray",
        alpha=0.3
    )
    fig.add_artist(con)
    connect_patch_list.append(con)

    part2_ax_list.append(temp_ax)
    annotate_axes(temp_ax, value)

y_label = temp_data['date'].dt.strftime('%m-%d').tolist()
part2_ax_list[-1].set_xticks(list(range(len(y_label))))
part2_ax_list[-1].set_xticklabels(y_label,rotation=45)


fig.suptitle('各个模块pv')
fig.savefig("test.png", dpi=300)